In [1]:
from FlagEmbedding import BGEM3FlagModel
import pandas as pd
import torch
import torch.nn as nn
from collections import defaultdict
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# Import learning rate scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import LightningModule
from sklearn.model_selection import KFold
import numpy as np

# Create embedded data

In [2]:
embedder = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
train_df = pd.read_csv("../data/train.csv", index_col="ID")
test_df = pd.read_csv("../data/test.csv", index_col="ID")

all_df = pd.concat([train_df, test_df], axis=0)

In [4]:
all_df

,set,question,answer,score
ID,,,,
0,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรจะมีค่าต่ำ เพราะว่าเราต้องการทร...,0.0
1,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะเป็นการเก็บข้อมูลจากหลาย users ...,5.0
2,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรเป็น #checkout events/ #cookies...,5.0
3,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,"เห็นด้วย ให้ X~Binomial(N,p) โดย p เป็นอัตราส่...",2.0
4,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เนื่องจากการทดสอบ A/B Testing เป็นวิธ...,4.5
...,...,...,...,...
447,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ไม่เห็นด้วยเพราะว่าถ้าเราต้องการวัดว่าผู้ใช้คล...,NaN
448,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,สมควรใช้ A/B testing เพราะการย้ายตำแหน่งของโฆษ...,NaN
449,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,- การใช้ A/B testing จะช่วยให้แฮมทาโร่มีข้อมูล...,NaN


In [ ]:
embeddings_data = defaultdict(list)

BATCH_SIZE = 64
answers = all_df["answer"].tolist()
for i in tqdm(range(0, len(answers), BATCH_SIZE), desc="Processing batches"):

    batch = answers[i : i + BATCH_SIZE]
    embedded_answers = embedder.encode(batch)['dense_vecs']
    for j, answer in enumerate(batch):
        embeddings_data[i + j] = torch.tensor(embedded_answers[j], dtype=torch.float32)

torch.save(embeddings_data, "../data/llm_embeds/bge-m3-answer-embeddings.pt")

Processing batches: 100%|██████████| 8/8 [01:14<00:00,  9.36s/it]


In [ ]:
embeddings_data = defaultdict(list)

for set_id in all_df["set"].unique():
    set_df = all_df[all_df["set"] == set_id]
    set_question = set_df["question"].iloc[0]
    embedding = embedder.encode([set_question])['dense_vecs'][0]
    embeddings_data[set_id] = torch.tensor(embedding, dtype=torch.float32)

torch.save(embeddings_data, "../data/llm_embeds/bge-m3-question-embeddings.pt")

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [41]:
del embedder

# Create a regressor

In [4]:
K_FOLDS = 10
BATCH_SIZE = 8

INPUT_SIZE = 2048
LAYER_NUM = 4
DROPOUT_RATE = 0.2

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-2

# Learning rate scheduler
LRS_PATIENCE = 3
LRS_FACTOR = 0.7

# Early stopping
ES_PATIENCE = 20

MAX_EPOCHS = 100

In [ ]:
# Load the data
train_df = pd.read_csv("../data/train.csv", index_col="ID")
question_embeddings = torch.load("../data/llm_embeds/bge-m3-question-embeddings.pt", weights_only=False)
answer_embeddings = torch.load("../data/llm_embeds/bge-m3-answer-embeddings.pt", weights_only=False)

In [6]:
# Assign fold to each row
train_df["fold"] = -1
for fold, (train_idx, val_idx) in enumerate(KFold(n_splits=K_FOLDS, shuffle=True, random_state=42).split(train_df)):
    train_df.loc[val_idx, "fold"] = fold

In [7]:
class BGEM3Dataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        question_embeddings: dict[int, torch.Tensor],
        answer_embeddings: dict[str, torch.Tensor],
        has_score: bool = True,
    ) -> None:
        self.df = df
        self.question_embeddings = question_embeddings
        self.answer_embeddings = answer_embeddings

        self.has_score = has_score

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Get the embeddings
        question_embedding = self.question_embeddings[row["set"]]
        answer_embedding = self.answer_embeddings[idx]
        input_embedding = torch.cat([question_embedding, answer_embedding], dim=0).type(torch.float32)

        if self.has_score:
            return input_embedding, row["score"].astype(np.float32)
        else:
            return input_embedding

In [8]:
class MLPRegressor(LightningModule):
    def __init__(self, input_size: int, layer_num: int, dropout_rate: float) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.LeakyReLU(),
            *[nn.Sequential(nn.Linear(1024, 1024), nn.LeakyReLU(), nn.Dropout(dropout_rate)) for _ in range(layer_num)],
            nn.Linear(1024, 1),
            nn.Sigmoid(),
        )

        self.loss_fn = nn.MSELoss(reduction="mean")

    def forward(self, x):
        return self.model(x) * 5.0

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

        # Log the learning rate
        self.log(
            "learning_rate",
            self.trainer.optimizers[0].param_groups[0]["lr"],
            on_step=True,
            on_epoch=False,
            prog_bar=True,
        )

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=LRS_FACTOR, patience=LRS_PATIENCE, verbose=True)
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss",
        }

In [9]:
models = [MLPRegressor(input_size=INPUT_SIZE, layer_num=LAYER_NUM, dropout_rate=DROPOUT_RATE) for _ in range(K_FOLDS)]
trainers = []
for current_fold in range(K_FOLDS):
    train_fold_df = train_df[train_df["fold"] != current_fold]
    val_fold_df = train_df[train_df["fold"] == current_fold]

    # Create the datasets and dataloaders
    train_dataset = BGEM3Dataset(train_fold_df, question_embeddings, answer_embeddings)
    val_dataset = BGEM3Dataset(val_fold_df, question_embeddings, answer_embeddings)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # Create logger
    logger = CSVLogger("logs/bgem3", name=f"bgem3-{current_fold}")

    # Create a checkpoint callback
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=f"./logs/bgem3/bgem3-{current_fold}/checkpoints",
        filename="bgem3-{epoch:02d}-{val_loss:.2f}",
        save_top_k=2,
        mode="min",
    )

    # Create an early stopping callback
    early_stopping = EarlyStopping(monitor="val_loss", patience=ES_PATIENCE, mode="min")

    # Create a trainer
    trainer = Trainer(max_epochs=MAX_EPOCHS, logger=logger, callbacks=[checkpoint_callback, early_stopping])

    # Fit the model
    trainer.fit(models[current_fold], train_loader, val_loader)

    # Append the trainer
    trainers.append(trainer)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/jirayuwat/anaconda3/envs/nlp/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/jirayuwat/Desktop/2110572-NLP/kaggle_midterm/mlp_topup/logs/bgem3/bgem3-0/checkpoints exists and is not empty.

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | model   | Sequential | 6.3 M  | train
1 | loss_fn | MSELoss    | 0      | train
-----------------------------------------------
6.3 M     Trainable params
0         Non-trainable params
6.3 M     Total params
25.190    Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/jirayuwat/anaconda3/envs/nlp/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/jirayuwat/anaconda3/envs/nlp/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/jirayuwat/anaconda3/envs/nlp/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (41) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [10]:
# Load the best model
best_models = []
for current_fold in range(K_FOLDS):
    model = MLPRegressor.load_from_checkpoint(
        trainers[current_fold].checkpoint_callback.best_model_path,
        input_size=INPUT_SIZE,
        layer_num=LAYER_NUM,
        dropout_rate=DROPOUT_RATE,
    )
    best_models.append(model)

# Create submission

In [11]:
test_df = pd.read_csv("../data/test.csv", index_col="ID")

test_dataset = BGEM3Dataset(test_df, question_embeddings, answer_embeddings, has_score=False)
test_loader = DataLoader(test_dataset, batch_size=4)

predictions = []
for model in best_models:
    model.eval()
    model.freeze()

    model_predictions = []
    for x in test_loader:
        y_hat = model(x.to("mps")).squeeze()
        model_predictions.extend(y_hat.tolist())

    predictions.append(model_predictions)

# Average the predictions
predictions = np.mean(predictions, axis=0)

test_df["score"] = predictions
test_df[["score"]].to_csv("bgem3_and_mlp_submission.csv")